In [1]:
import pandas as pd

def df_from_sql(sql):
    import pymysql
    import pandas as pd
    connect = pymysql.Connect(
        host=polar_host,
        port=polar_port,
        user=polar_user,
        passwd=polar_pswd,
        db=polar_db
    )
    cursor = connect.cursor()
    df = pd.read_sql(sql, con=connect)
    connect.close()    
    return df

In [ ]:
# 只要7月没有店铺数据，d就扔掉店铺全部交易

In [2]:
# 虹如要的数据；
# 1. reblue全量 7月 - 全量 6月；

sql_loan = '''
select * 
from yk_reblue_sale
'''

blue = df_from_sql(sql_loan)

june_blue = blue[blue.sale_time.between('2020-06-01 00:00', '2020-07-01 00:00')]
july_blue = blue[blue.sale_time.between('2020-07-01 00:00', '2020-08-01 00:00')]

print('7月总交易额{}'.format(july_blue.paid_amt.sum()))
print('6月总交易额{}'.format(june_blue.paid_amt.sum()))
print('7-6月 :{}'.format(july_blue.paid_amt.sum() - june_blue.paid_amt.sum()))


7月总交易额5036275.37
6月总交易额4153047.32
7-6月 :883228.0500000003


In [4]:
# 虹如要的数据；
# 2. reblue全量 7月 客单价 总价/人数，如果非会员用dp_id代表人数；
sql_loan = '''
select * 
from yk_reblue_sale s
left join yk_reblue_user u on u.user_id = s.user_id
'''

blue = df_from_sql(sql_loan)
june_blue = blue[blue.sale_time.between('2020-06-01 00:00', '2020-07-01 00:00')]
july_blue = blue[blue.sale_time.between('2020-07-01 00:00', '2020-08-01 00:00')]

july_blue = july_blue[['user_id', 'paid_amt', 'dp_id', 'xp_id']]
june_blue = june_blue[['user_id', 'paid_amt', 'dp_id', 'xp_id']]

june_blue = june_blue[june_blue.paid_amt>0].iloc[:,1:]
print('6月大票数：{}'.format(june_blue.dp_id.nunique()))
print('6月单均：{}'.format(june_blue.paid_amt.sum()/june_blue.dp_id.nunique()))

july_blue = july_blue[july_blue.paid_amt>0].iloc[:,1:]
print('7月大票数:{}'.format(july_blue.dp_id.nunique()))
print('7月单均:{}'.format(july_blue.paid_amt.sum()/july_blue.dp_id.nunique()))


6月大票数：3413
6月单均：1446.8952475827718
7月大票数:3934
7月单均:1474.9359405185564


In [87]:
# 虹如要的数据；
# 3. reblue全量 5.1~6.16 交易额/天数
sql_loan = '''
select * 
from yk_reblue_sale s
'''
blue = df_from_sql(sql_loan)
tmp_blue = blue[blue.sale_time.between('2020-05-01 00:00:00', '2020-06-17 00:00:00')]
print('reblue全量 5.1~6.16 交易额/天数:{}'.format(tmp_blue.paid_amt.sum()/47))

reblue全量 5.1~6.16 交易额/天数:140410.93914893616


In [88]:
# 虹如要的数据；
# 4. reblue全量 周数据[首周22-28]；每周交易额
# 5. reblue西瓜 周数据[首周22-28]；每周交易额

sql_loan = '''
select * 
from yk_reblue_sale s
'''
blue = df_from_sql(sql_loan)
tmp_blue = blue[blue.sale_time.between('2020-06-22 00:00:00', '2020-08-01 00:00:00')]
tmp_blue = tmp_blue[['paid_amt', 'hapay_pay', 'dp_id', 'user_id', 'sale_time']]
tmp_blue['date'] = tmp_blue['sale_time'].apply(lambda x:str(x)[0:10])

def weekget(string):
    if string in ['2020-06-22','2020-06-23','2020-06-24','2020-06-25','2020-06-26','2020-06-27','2020-06-28']:
        return '上线第1周'
    elif string in ['2020-06-29','2020-06-30','2020-07-01','2020-07-02','2020-07-03','2020-07-04','2020-07-05']:
        return '上线第2周'
    elif string in ['2020-07-06','2020-07-07','2020-07-08','2020-07-09','2020-07-10','2020-07-11','2020-07-12']:
        return '上线第3周'
    elif string in ['2020-07-13','2020-07-14','2020-07-15','2020-07-16','2020-07-17','2020-07-18','2020-07-19']:
        return '上线第4周'    
    elif string in ['2020-07-20','2020-07-21','2020-07-22','2020-07-23','2020-07-24','2020-07-25','2020-07-26']:
        return '上线第5周'    
    elif string in ['2020-07-27','2020-07-28','2020-07-29','2020-07-30','2020-07-31']:
        return '上线第6周'    
    else:
        print(string)
        raise Exception('not allowed')

tmp_blue['week'] = tmp_blue.date.apply(lambda x: weekget(x))
tmp_blue.pivot_table(index='week', values=['paid_amt', 'hapay_pay'], aggfunc=sum)/7

,hapay_pay,paid_amt
week,,
上线第1周,28241.157143,155995.300000
上线第2周,87218.892857,150242.352857
上线第3周,71871.778571,158158.557143
上线第4周,60075.800000,158836.867143
上线第5周,85485.885714,196675.012857
上线第6周,30851.292857,82636.914286


In [89]:
# 虹如要的数据；
# 6. reblue西瓜 新老顾客
# 西瓜新老用户定义：7月1号之后来的，之前没来过都是新用户
sql_loan = '''
select * 
from mars_financial_settle_details f
left join mars_hapay_store s on s.refid = f.storerefid
left join mars_loan l on l.refid = f.loanrefid
left join mars_user u on u.refid = l.userrefid
where f.type='交易'
'''

blue = df_from_sql(sql_loan)

blue = blue[blue.financeRefId.isin(['WFY2_Re-BLUE_default'])]
blue['flag'] = 1
user = blue.pivot_table(index='userRefId', 
                        values=['goodsOrderAmount', 'flag'],
                        aggfunc={'goodsOrderAmount':sum,
                                 'flag':sum})


old_users = blue[blue.fundTime<'2020-07-01 00:00'].userRefId
july_blue = blue[blue.fundTime>'2020-07-01 00:00']
july_blue['是老用户'] = july_blue.userRefId.isin(old_users)
# july_blue.index='userRefId', values='是老用户', aggfunc='')
h = july_blue[['userRefId', '是老用户']].drop_duplicates().set_index('userRefId')
h['是老用户'].value_counts()


/Users/cashbus/py3env/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


False    263
True      15
Name: 是老用户, dtype: int64

In [90]:
# 虹如要的数据；
# 7. reblue 门店 月份 西瓜 ---- 总量 单数 金额;
sql_loan = '''
select * 
from yk_reblue_sale s
left join yk_reblue_user u on u.user_id = s.user_id
left join yk_reblue_saler sr on sr.saler_id = s.saler_id
'''
blue = df_from_sql(sql_loan)
tmp_blue = blue[blue.sale_time.between('2020-06-01 00:00:00', '2020-08-01 00:00:00')]
tmp_blue = tmp_blue[['paid_amt', 'hapay_pay', 'dp_id', 'user_id', 'sale_time', 'store_name']]
tmp_blue['month'] = tmp_blue['sale_time'].apply(lambda x:str(x)[5:7])

show1 = tmp_blue.pivot_table(index=['month', 'store_name'], 
                             values=['paid_amt','dp_id'], 
                             aggfunc={'paid_amt':sum, 'dp_id':'nunique'})

xg = tmp_blue[tmp_blue.hapay_pay>0]
xg = xg[['dp_id', 'hapay_pay', 'store_name', 'month']]
xg = xg.drop_duplicates()
show2 = xg.pivot_table(index=['month', 'store_name'],
                       values=['hapay_pay','dp_id'], 
                       aggfunc={'hapay_pay':sum, 'dp_id':'nunique'})

In [ ]:
show1

In [92]:
show2

dp_id  hapay_pay
month store_name                  
06    上海宝杨宝龙          3    2517.20
      兰州七里河店          8   14614.20
      北京九龙山合生汇        8   13025.90
      北京国瑞城           8   13225.80
      北京大兴荟聚中心店       2    1606.90
      北京朝阳金地中心店       7    8970.15
      南京华采天地          1     279.30
      南京龙湾天街          5    4524.00
      常州环球港           1     509.70
      杭州滨江宝龙城店        4    3664.70
      杭州滨江龙湖天街        3    4185.30
      杭州紫荆龙湖          7   12275.00
      杭州西溪印象城         1     939.20
      江苏南京印象汇         1    1040.00
      江苏苏州工业园         3    6269.70
      江苏苏州昆山九方城       1    1039.20
      河南郑州永威木色        2    1697.80
      济南恒隆            1    3986.30
      深圳福田卓悦汇        13   21131.85
      西安大融城           1    3526.10
07    上海宝杨宝龙         21   39844.20
      云南南亚风情城         1    4313.40
      兰州七里河店         70  103600.45
      北京九龙山合生汇       53   89632.15
      北京国瑞城          30   57611.95
      北京大兴荟聚中心店      31   53728.80
      北京朝阳金地中心店      17   31059.85
      南京中海环宇城         3    5235.10
      南京龙湾天街          2    1398.80
      呼和浩特振华店         7   15380.50
      四川成都珠江广场        2    1857.80
      四川成都龙湖西宸天街      1     109.30
      杭州乐堤港           7   12346.10
      杭州滨江宝龙城店        6    8597.00
      杭州滨江龙湖天街        7   15932.70
      杭州紫荆龙湖         28   63297.80
      杭州西溪印象城         1    1299.00
      江苏苏州工业园         4   11459.80
      河南郑州公园茂店        4   30897.00
      济南恒隆            5   10198.80
      深圳福田卓悦汇        21   33046.00
      石家庄万达           2    2177.90
      西安大融城          13   27763.15
      西安曲江新区金地        3    5995.40
      贵阳金阳南路店         8   12157.70

In [97]:
# 游总要的数据 - 周-门店-金额-单数；

sql_wxtd = '''
select * 
from mars_financial_settle_details f
left join mars_hapay_store s on s.refid = f.storerefid
left join mars_loan l on l.refid = f.loanrefid;
'''
f = df_from_sql(sql_wxtd)
filter1 = '''KECHUAN_SZWXTD_default
IMPOS_SZWXTD_default
XGMDSY_CC_drivepro'''.splitlines()
sw = f[f.financeRefId.isin(filter1)]
sww = sw[['created', 'loanRefId', 'type', 'goodsOrderAmount', 'storeRefId', 'name', 'financeRefId', ]]
sww.columns = ['created', 'created1', 'created2', 'loanRefId', 'type', 'type1', 'type2',
       'goodsOrderAmount', 'storeRefId', 'name', 'name1', 'financeRefId']
refund_loans = sww[sww.type=='退款'].loanRefId

sww2 = sww[~sww.loanRefId.isin(refund_loans)]

sww3 = sww2[['created', 'loanRefId', 'goodsOrderAmount', 'name1', 'financeRefId']].copy()

sww3['date'] = sww3.created.apply(lambda x: str(x)[5:10])
def apply_week(date_str):
    if date_str in ['07-17', '07-18', '07-19', '07-20', '07-21', '07-22', '07-23']:
        return '上线第一周'
    elif date_str in ['07-24', '07-25', '07-26', '07-27', '07-28', '07-29', '07-30']:
        return '上线第二周'    
    else:
        raise Exception('not allowed!')
sww3['week'] = sww3['date'].apply(lambda x: apply_week(x))

# sww3.to_csv('/Users/cashbus/Desktop/深圳万象天地两周原始数据4.csv', float_format='%.1f')
sww3['num_orders']= 1

sww4 = sww3.pivot_table(index=['week','name1'], 
                 values=['goodsOrderAmount','num_orders'], 
                 aggfunc={'goodsOrderAmount':sum, 'num_orders':sum})


all_stores = df_from_sql('select * from mars_hapay_store')
all_stores = all_stores[all_stores.financeRefId.isin(filter1)].name.unique()

res = []
for week in ['上线第一周', '上线第二周']:
    tmp = sww4.loc[week]
    for s in all_stores:
        try:
            tmp.loc[s]
        except:
            tmp.loc[s, :] = 0, 0 
    tmp['week'] = week
    res.append(tmp)

final = pd.concat(res)

In [88]:
# 虹如要的数据；
# 1. hapay-reblue 价格分布；

sql_loan = '''
select * 
from mars_financial_settle_details f
left join mars_hapay_store s on s.refid = f.storerefid
left join mars_loan l on l.refid = f.loanrefid
left join mars_user u on u.refid = l.userrefid
where f.type='交易'
'''

blue = df_from_sql(sql_loan)
blue = blue[blue.financeRefId.isin(['WFY2_Re-BLUE_default'])]
july_blue = blue[blue.fundTime>'2020-07-01 00:00']
user = july_blue.pivot_table(index='userRefId', values=['goodsOrderAmount'], aggfunc=sum)
amt_dist = pd.cut(user.goodsOrderAmount, 
       bins=[0,1000,2000,3000,4000,5000,6000,1000000], 
       labels=['￥0-1000', '￥1000-2000', '￥2000-3000', '￥3000-4000', '￥4000-5000', '￥5000-6000', '￥6000以上']
      )

pd.DataFrame(amt_dist.value_counts()).to_csv('/Users/cashbus/Desktop/按人交易额分布2.csv')

In [19]:
# 虹如要的数据；
# 2. reblue回头客统计；
# 西瓜新老用户定义：7月1号之后来的，之前没来过都是新用户;

sql_loan = '''
select * 
from mars_financial_settle_details f
left join mars_hapay_store s on s.refid = f.storerefid
left join mars_loan l on l.refid = f.loanrefid
left join mars_user u on u.refid = l.userrefid
where f.type='交易'
'''

blue = df_from_sql(sql_loan)

blue = blue[blue.financeRefId.isin(['WFY2_Re-BLUE_default'])]
blue['flag'] = 1
user = blue.pivot_table(index='userRefId', 
                        values=['goodsOrderAmount', 'flag'],
                        aggfunc={'goodsOrderAmount':sum,
                                 'flag':sum})



old_users = blue[blue.fundTime<'2020-07-01 00:00'].userRefId
july_blue = blue[blue.fundTime>'2020-07-01 00:00']
july_blue['是老用户'] = july_blue.userRefId.isin(old_users)
# july_blue.index='userRefId', values='是老用户', aggfunc='')
h = july_blue[['userRefId', '是老用户']].drop_duplicates().set_index('userRefId')
h['是老用户'].value_counts()

/Users/cashbus/py3env/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


False    261
True      14
Name: 是老用户, dtype: int64

In [18]:
# 虹如要的数据；
# 3. reblue 7月份 年龄 人均消费情况；
# 西瓜新老用户定义：7月1号之后来的，之前没来过都是新用户
sql_loan = '''
select * 
from mars_financial_settle_details f
left join mars_hapay_store s on s.refid = f.storerefid
left join mars_loan l on l.refid = f.loanrefid
left join mars_user u on u.refid = l.userrefid
left join mars_person p on p.refid = u.personrefid
where f.type='交易'
'''

blue = df_from_sql(sql_loan)

blue = blue[blue.financeRefId.isin(['WFY2_Re-BLUE_default'])]
july_blue = blue[blue.fundTime>'2020-07-01 00:00']
july_blue['age'] = (pd.to_datetime('2020-07-31') - pd.to_datetime(july_blue.birthdate))/pd.Timedelta('365 days')


jb = july_blue[['fundTime', 'goodsOrderAmount', 'age', 'personRefId']].iloc[:,:-1]
jb['年龄分段'] = pd.cut(jb.age, bins=[0,20,30,40,50,120], labels=['20岁以下', '21~30岁', '31~40岁', '41~50岁', '50岁以上'])
jb.pivot_table(index=['年龄分段'], values=['goodsOrderAmount', 'personRefId'], aggfunc={'goodsOrderAmount':sum,
                                                                                   'personRefId':'nunique'})

/Users/cashbus/py3env/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,goodsOrderAmount,personRefId
年龄分段,,
20岁以下,0.00,0
21~30岁,201285.35,104
31~40岁,338476.20,137
41~50岁,88220.30,28
50岁以上,10958.80,6


In [36]:
# 虹如要的数据；
# 4. reblue全量 7月 - 全量 6月；

sql_loan = '''
select * 
from yk_reblue_sale
'''

blue = df_from_sql(sql_loan)

june_blue = blue[blue.sale_time.between('2020-06-01 00:00', '2020-07-01 00:00')]
july_blue = blue[blue.sale_time.between('2020-07-01 00:00', '2020-08-01 00:00')]

july_blue.paid_amt.sum()
june_blue.paid_amt.sum()
july_blue.paid_amt.sum() - june_blue.paid_amt.sum()


883228.0500000003

In [40]:
july_blue.paid_amt.sum()

5036275.37

In [34]:
# 虹如要的数据；
# 5. reblue全量 7月 客单价 总价/人数，如果非会员用dp_id代表人数；

sql_loan = '''
select * 
from yk_reblue_sale s
left join yk_reblue_user u on u.user_id = s.user_id
'''

blue = df_from_sql(sql_loan)
july_blue = blue[blue.sale_time.between('2020-07-01 00:00', '2020-08-01 00:00')]


t = july_blue[['user_id', 'paid_amt', 'dp_id', 'xp_id']]
t = t[t.paid_amt>0].iloc[:,1:]

novip = t[t.user_id.isnull()]
vip = t[~t.user_id.isnull()]
print(novip.dp_id.nunique())
print(vip.user_id.nunique())

t.paid_amt.sum()/(novip.dp_id.nunique() + vip.user_id.nunique())

546
2512


1897.4486559843037

In [ ]:
# 6. rebl 

In [35]:
# 全用dp
print(t.dp_id.nunique())
print(t.paid_amt.sum()/t.dp_id.nunique())

3934
1474.9359405185564


In [22]:
t.pivot_table(index=['user_id'], 
              values=['paid_amt', 'dp_id'], 
              aggfunc={'paid_amt':sum, 'dp_id':'nunique'})


,dp_id,paid_amt
user_id,,
00020006,41,61603.94
00060002,1,1608.60
13002212195,1,719.20
13011031939,1,349.50
13018965881,1,619.20
...,...,...
EZR00000000000003321,2,2078.30
EZR00000000000003322,1,1118.40
EZR00000000000003323,1,1738.40


In [2]:
def schema_from_sql(tb_name, to_type='pg'):
    #1. 从polar/sql中抽取表结构;
    '''
    tb_name: table name;
    to_type: 'pg', 'odps'
    '''
    assert to_type in ['pg', 'odps']
    import pymysql
    connect = pymysql.Connect(
        host=polar_host,
        port=polar_port,
        user=polar_user,
        passwd=polar_pswd,
        db=polar_db
    )
    cursor = connect.cursor()
    query_get_one_schema = '''SHOW COLUMNS FROM {};'''.format(tb_name)
    cursor.execute(query_get_one_schema)
    res = cursor.fetchall()
    table_schema_df = pd.DataFrame(res, columns = ['col', 'type', 'null', 'K', 'default', 'extra'])
    #print(table_schema_df)
    if to_type == 'pg':
        table_schema_df['type'] = table_schema_df.type.apply(str2pg_type)
    elif to_type == 'odps':
        table_schema_df['type'] = table_schema_df.type.apply(str2odps_type)
    else:
        raise Exception('not allowed')
    return table_schema_df

In [10]:
from pytube import YouTube
VEDIO = YouTube('https://www.youtube.com/watch?v=JRA2lxuLF3Y&frags=wn')

In [11]:
VEDIO.streams[0].download()

'/Users/cashbus/Desktop/YouTube.mp4'

In [1]:
from colormap import rgb2hex, hex2rgb
COLORS = {}
COLORS['green'] = rgb2hex(211, 230, 196)
COLORS['blue'] = rgb2hex(203, 216, 239)
COLORS['yellow'] = rgb2hex(255, 248, 230)
COLORS['red'] = rgb2hex(254, 173, 169)
COLORS['black'] = rgb2hex(133, 143, 160)

In [107]:
from odps import ODPS
from collections import defaultdict
import json
from odps import options
import pandas as pd
from odps.df import DataFrame
import ipdb
import xlsxwriter
o = ODPS('LTAI4GK2qnyJEcGaTQBngnCA', 'r0W0kodzenHTyDQG8WoIEPTiPddJSv', 'xianmai_dw_dev', 'http://service.cn-beijing.maxcompute.aliyun.com/api')

# options.verbose=True
tmp = DataFrame(o.get_table('xianmai_last_record_ps2')).to_pandas()

TransientProgressBar(value=0.0)

InstancesProgress()

HTMLNotifier()

In [3]:
tmp

,table_name,lasttime,lastrecord,tablename
0,mars_hapay_refund_task,2020-07-03 00:00:00,1,mars_hapay_refund_task
1,mars_loan,2020-07-03 00:00:00,1,mars_loan
2,mars_loan_meta,2020-07-03 00:00:00,1,mars_loan_meta
3,mars_wy_event,2020-07-03 00:00:00,1,mars_wy_event
4,mongo_credit,2020-07-03 00:00:00,1,mongo_credit
5,mongo_credit_loaned,2020-07-03 00:00:00,1,mongo_credit_loaned
6,mongo_loan,2020-07-03 00:00:00,1,mongo_loan
7,mongo_process,2020-07-03 00:00:00,1,mongo_process


In [117]:
mgg_credit = DataFrame(o.get_table('pg_mongo_credit')).to_pandas()
mgg_loan = DataFrame(o.get_table('pg_dwd_mongo_loan')).to_pandas()
mgg_credit_loaned = DataFrame(o.get_table('pg_dwd_mongo_credit_loaned')).to_pandas()
mgg_process = DataFrame(o.get_table('pg_dwd_mongo_process')).to_pandas()

mss_loan = DataFrame(o.get_table('pg_dwd_mars_loan')).to_pandas()
mss_device = DataFrame(o.get_table('pg_dwd_mars_device')).to_pandas()
mss_loan_meta = DataFrame(o.get_table('pg_dwd_mars_loan_meta')).to_pandas()
mss_wy = DataFrame(o.get_table('pg_dwd_mars_wy_event')).to_pandas()
mss_refund = DataFrame(o.get_table('pg_dwd_mars_hapay_refund_task')).to_pandas()

TransientProgressBar(value=0.0)

InstancesProgress()

HTMLNotifier()

TransientProgressBar(value=0.0)

InstancesProgress()

...

InstancesProgress()

HTMLNotifier()

TransientProgressBar(value=0.0)

InstancesProgress()

HTMLNotifier()

In [119]:
mss_loan = DataFrame(o.get_table('pg_dwd_mars_loan')).to_pandas()
ms_loan2 = mss_loan[['refid', 'status', 'fundtime']].drop_duplicates()
ms_loan2[(ms_loan2.status.isin(['Funded', 'PaidOff']))&(ms_loan2.fundtime.between('2020-06-18 00:00:00', '2020-07-16 00:00:00'))]

TransientProgressBar(value=0.0)

InstancesProgress()

HTMLNotifier()

,refid,status,fundtime
272,Sj6C24_CNHSEnHl,Funded,2020-06-18 13:27:08
282,Vecpon_CNjCOwMm,Funded,2020-06-18 22:01:39
288,NM6yeT_CNpKRZdP,Funded,2020-06-19 18:34:03
289,iHXjKz_CNtrNEVR,Funded,2020-06-19 19:00:37
291,za8uSl_CNJYcchg,Funded,2020-06-19 21:47:14
...,...,...,...
688,7y2pmu_CNCPANbd,Funded,2020-07-09 19:08:21
689,4aM3BC_CNZgowk6,Funded,2020-07-09 19:10:51
690,AGeVs0_CNQzC4X8,Funded,2020-07-09 19:12:06
696,qe2sXA_CNJAlpT2,Funded,2020-07-10 12:20:55


In [ ]:
# 检查1： marsLoan表在618-716之间funed,paidoff共219笔

In [120]:
# MARs
#------------------------------------------------------------------------------------------------------------------------------------------------------

## 1. meta表处理; [loanid, 品牌, 件数, 总价， 描述];
ms_loan_meta2 = mss_loan_meta[['loanrefid', 'brandid', 'storeid', 'storename', 'goodstotalnum', 'goodsprice', 'orderdesc']]
ms_loan_meta2 = ms_loan_meta2[~ms_loan_meta2.brandid.isin(['prodDemoBrand', None])]

## 2. mars_loan表处理;  [状态, 放款时间];
ms_loan2 = mss_loan[['refid', 'status', 'fundtime']]
mongo_loan2 = mgg_loan[['credit_id', 'loan_id']].drop_duplicates()
ms_loan2 = ms_loan2[ms_loan2.refid.isin(mongo_loan2.loan_id)] # 以mongo_loan为准；
# ms_loan2 = ms_loan2[ms_loan2.status.isin(['Funded', 'PaidOff'])]
# ms_loan2 = ms_loan2[~ms_loan2.status.isin(['Canceled', 'Refund'])]

# 3. wy表 [credit_id, loan_id];
ms_wy2 = mss_wy[['userrefid', 'loanrefid', 'eventid']]
ms_wy2 = ms_wy2[ms_wy2.eventid.isin(['70'])] 

# 4. device;
ms_device2 = mss_device[['userrefid', 'channel']]

# 5. 汇总
sale_df = ms_loan2.merge(ms_loan_meta2, 
                         how='left', 
                         left_on='refid', 
                         right_on='loanrefid').merge(ms_wy2,how='left').merge(ms_device2,how='left')
sale = sale_df
# sale = sale_df[sale_df.status.isin(['Funded', 'PaidOff'])]


# Mongo
#------------------------------------------------------------------------------------------------------------------------------------------------------
## 1. mongo_loan;
mg_loan2 = mgg_loan[['credit_id', 'loan_id']].drop_duplicates()

## 2. mg_credit;
usr_df = mgg_credit[['name', 'mobile', 'gender', 'idcard_no', 'credit_id', 'credit_time', 'update_time','gps_city','amount','guarantor_name','guarantor_mobile']]

## 3. process 表; 
loan_ids_in_mp = mgg_process[(mgg_process.var_name=='事件类型') & (mgg_process.var_value=='53')]._id.tolist()
mg_process2 = mgg_process[mgg_process._id.isin(loan_ids_in_mp)]
credit_process_df = mg_process2.pivot(index='credit_id', columns='var_name', values=['var_value'])
credit_process_df.columns = [i[1] for i in credit_process_df.columns]
credit_process_df = credit_process_df.reset_index()
credit_process_df2 = credit_process_df[['credit_id', '申请人手机号', '申请人手机号所属城市','申请人身份证城市', '申请人年龄','营业员ID', 
 '信用分', '通用模型分', '通用模型&信用分二维分层', '推荐人欺诈分', '担保人手机号',
 '信用卡还款金额', '申请人1年内公务舱飞行次数', '申请人1年内头等舱飞行次数', '申请人1年内经济舱飞行次数', '申请人EZA INMIX会员最近3个月消费金额',
       '申请人EZA INMIX会员最近3个月退货金额', '申请人EZA RE-BLUE会员最近3个月消费金额',
       '申请人EZA RE-BLUE会员最近3个月退货金额', '申请人INMIX会员最近6个月消费金额',
       '申请人INMIX会员最近6个月退货金额', '申请人INMIX会员等级', '申请人INMIX入会天数', '申请人RE-BLUE会员等级',
       '申请人RE-BLUE入会天数', '申请人Re-BLUE会员最近6个月消费金额', '申请人Re-BLUE会员最近6个月退货金额',
       '申请人借记卡360天交易金额', '申请人借记卡360天信用卡还款', '申请人借记卡360天消费金额', '申请人在INMIX最近1年累计消费金额',
       '申请人在INMIX最近1年累计退货金额','申请人在Re-BLUE最近1年累计消费金额', '申请人在Re-BLUE最近1年累计退货金额',]]
credit_process_df2 = credit_process_df2.reset_index()

## 4. credit_loaned
tie_df = mgg_credit_loaned.pivot_table(index='credit_id', columns=['var_name'], values=['var_value'], aggfunc=max)
tie_df.columns = [i[1] for i in tie_df.columns]
tie_df2 = tie_df[['申请人12个月内乘坐高端席别比例超过70%', '申请人1年内公务舱飞行次数', '申请人1年内头等舱飞行次数',
       '申请人1年内经济舱飞行次数', '申请人GDC历史乘车次数8+次', '申请人信用卡360天消费金额', '申请人信用卡360天还款金额',
       '申请人借记卡360天交易金额', '申请人借记卡360天信用卡还款', '申请人借记卡360天消费金额',
       '申请人手机号作为申请人存在当前逾期的个数', '申请人是否有3次+高端商旅席别记录', '申请人近3月乘车5+次']]
tie_df2 = tie_df2.reset_index()

## 5. 汇总
user = usr_df.merge(credit_process_df2, how='left').merge(tie_df2, how='left')

# final
#------------------------------------------------------------------------------------------------------------------------------------------------------
final = sale.merge(mg_loan2, how='left', left_on='loanrefid', right_on='loan_id').merge(user, how='outer', left_on='credit_id', right_on='credit_id')
# fff = final[(final.brandid=='Re-BLUE')]
fff = final[(final.brandid=='INMIX')]


In [121]:
# f1 = final[final.brandid.isin(['Re-BLUE', 'INMIX'])]

In [122]:
f1.status.value_counts()

Funded      147
Refund       81
Canceled     66
PaidOff      10
Name: status, dtype: int64

In [123]:
final[final.status.isin(['Funded', 'PaidOff'])].shape

(161, 66)

In [124]:
from colormap import rgb2hex, hex2rgb
COLORS = {}
COLORS['green'] = rgb2hex(211, 230, 196)
COLORS['blue'] = rgb2hex(203, 216, 239)
COLORS['yellow'] = rgb2hex(255, 248, 230)
COLORS['red'] = rgb2hex(254, 173, 169)
COLORS['black'] = rgb2hex(133, 143, 160)

def prepare_day_format(df):
    days=pd.to_datetime(pd.Series(df.index),format='%Y-%m-%d')
    week = '第'+days.dt.weekofyear.map(str)+'周'
    day_name = days.dt.month.map(str)+'月'+days.dt.day.map(str) + '日'
    df['new_index'] = day_name.tolist()
    df = df.sort_index().set_index('new_index')
    return df


def df2charts(df, excel_pth, sheetname, color_list, chart_type, index_name, gridline_type, format_list, header_rownum=1, workbook=None,worksheet=None, add_chart=True, add_chart_style='new'):
    assert df.shape[1] == len(color_list) == len(format_list), [df.columns,color_list, format_list]
    # 数据写进去
    if not workbook:
        workbook = xlsxwriter.Workbook(excel_pth)
    format_list = [workbook.add_format({'num_format': i}) for i in format_list]
#     format1 = workbook.add_format({'num_format': '#,##0.00'})
#     format2 = workbook.add_format({'num_format': '0.0%'})
    if not worksheet:
        worksheet = workbook.add_worksheet(name=sheetname)
    bold = workbook.add_format({'bold': 1})
    headings = [index_name] + list(df.columns)
    worksheet.write_row('A{}'.format(header_rownum), headings, bold)
    worksheet.write_column('A{}'.format(header_rownum+1), df.index.values)
    tmp = ['B{}'.format(header_rownum+1), 
           'C{}'.format(header_rownum+1),
           'D{}'.format(header_rownum+1),
           'E{}'.format(header_rownum+1), 
           'F{}'.format(header_rownum+1), 
           'G{}'.format(header_rownum+1),
           'H{}'.format(header_rownum+1),
           'I{}'.format(header_rownum+1)]
    for idx, _ in enumerate(df.columns):
        worksheet.write_column(tmp[idx], df.iloc[:,idx].values, cell_format=format_list[idx])
    if add_chart:
        # 每一列创建一个新图表写进去,这样一整个df可以对应多个charts
        if add_chart_style == 'new':
            print('一个df多个chart')
            for i in range(df.shape[1]):
                col_num = i+1
                chart1 = workbook.add_chart({'type': chart_type})
                chart1.add_series({
                                    'name':       [sheetname, 0, col_num],
                                    'categories': [sheetname, 1, 0, df.shape[0], 0],
                                    'values':     [sheetname, 1, col_num, df.shape[0], col_num],
                                    'fill':        {'color': COLORS[color_list[i]]},
                })
                chart1.set_style(3)
                chart1.set_legend({'position':'none'})
                if gridline_type:
                    chart1.set_y_axis({'major_gridlines': {'visible': True, 'dash_type': gridline_type}})
                else:
                    chart1.set_y_axis({'major_gridlines': {'visible': False}})
                worksheet.insert_chart('G{}'.format(col_num), chart1, {'x_offset': 25, 'y_offset': 10})
        # 一整个df只有一个堆叠chart;
        else:
            print('一个df一个chart')
            chart1 = workbook.add_chart({'type': chart_type})
            for i in range(df.shape[1]):
                col_num = i+1
                chart1.add_series({
                                    'name':       [sheetname, 0, col_num],
                                    'categories': [sheetname, 1, 0, df.shape[0], 0],
                                    'values':     [sheetname, 1, col_num, df.shape[0], col_num],
                                    'fill':        {'color': COLORS[color_list[i]]},
                })
            chart1.set_style(3)
            chart1.set_legend({'position':'none'})
            if gridline_type:
                chart1.set_y_axis({'major_gridlines': {'visible': True, 'dash_type': gridline_type}})
            else:
                chart1.set_y_axis({'major_gridlines': {'visible': False}})
            worksheet.insert_chart('G{}'.format(col_num), chart1, {'x_offset': 25, 'y_offset': 10})
        # 关闭文件
    return workbook, worksheet

def apply_week(row):
    if row in ['2020-06-18','2020-06-19','2020-06-20','2020-06-21','2020-06-22','2020-06-23','2020-06-24']:
        return '第一周'
    elif row in ['2020-06-25','2020-06-26','2020-06-27','2020-06-28','2020-06-29','2020-06-30','2020-07-01']:
        return '第二周'
    elif row in ['2020-07-02','2020-07-03','2020-07-04','2020-07-05','2020-07-06','2020-07-07','2020-07-08']:
        return '第三周'
    elif row in ['2020-07-09','2020-07-10','2020-07-11','2020-07-12','2020-07-13','2020-07-14','2020-07-15']:
        return '第四周'
    else:
        print(row)
        raise Exception('date not allowed!')

In [125]:
from colormap import rgb2hex, hex2rgb
COLORS = {}
COLORS['green'] = rgb2hex(211, 230, 196)
COLORS['blue'] = rgb2hex(203, 216, 239)
COLORS['yellow'] = rgb2hex(255, 248, 230)
COLORS['red'] = rgb2hex(254, 173, 169)
COLORS['black'] = rgb2hex(133, 143, 160)

PARAS = {
    'excel_pth':'/Users/cashbus/Desktop/西瓜买单月报_Inmix.xlsx',
    'store_excel_pth':'/Users/cashbus/Desktop/西瓜买单月报_Inmix-分门店数据.xlsx'
}

### 1. 每天交易额，每天单数，每天单均价；

In [126]:
# 1. 每天交易额，每天单数，每天单均价;
fff = fff[~fff.status.isin(['Canceled', 'Refund'])] 
fff['fund_dt'] = fff.fundtime.astype(str).apply(lambda x: x[:10])
fff = fff[fff.fund_dt>='2020-06-18']
tmp1 = fff[['loanrefid', 'goodsprice', 'idcard_no', 'name', 'gender', 'storename', 'status', 'fund_dt', '申请人年龄', '申请人INMIX入会天数']].copy().fillna(-111).astype({'goodsprice':'float64',
                                                                                            '申请人INMIX入会天数':'int',
                                                                                            '申请人年龄':'int'})
tmp1 = tmp1.drop_duplicates()
tmp1['num_orders']= 1
# days=pd.to_datetime(pd.Series(fff.fund_dt),format='%Y-%m-%d')
# week = '第'+days.dt.weekofyear.map(str)+'周'
# day_name = days.dt.month.map(str)+'月'+days.dt.day.map(str) + '日'
tmp1['week'] = fff.fund_dt.apply(lambda x: apply_week(x))
tmp1['fund_dt2'] = day_name
res1 = tmp1.pivot_table(index='fund_dt', values=['goodsprice','num_orders'], aggfunc=sum)
res1['mean'] = res1.goodsprice/res1.num_orders
res1.columns = ['金额', '单数', '每单金额']

wb, _ = df2charts(prepare_day_format(res1), 
               PARAS['excel_pth'], 
               '日数据',
               ['green', 'blue', 'yellow'],
               chart_type = 'area',
               index_name = '日期',
               gridline_type = 'dash',
               format_list=['#,##0.00', '0', '#,##0.00'])

一个df多个chart


In [127]:
a = tmp1.loanrefid.nunique()

b = tmp1.goodsprice.sum()

c = tmp1.goodsprice.sum()/tmp1.loanrefid.nunique()

d = tmp1[tmp1['申请人INMIX入会天数']<=0].idcard_no.nunique()
print('{}\n{}\n{}\n{}'.format(a,b,c,d))

5
6991.0
1398.2
3


## 2. 每周交易额，每天单数，每天单均;

In [65]:
# 2. 每周交易额，每天单数，每天单均;
res2 = tmp1.pivot_table(index='week', values=['goodsprice','num_orders'], aggfunc=sum)
res2['mean'] = res2.goodsprice/res2.num_orders
res2.columns = ['金额', '笔数', '每单金额']
res2.index = [i[0] + str(idx+1) + i[-1] for idx, i in enumerate(res2.index)]

In [66]:
wb, _ = df2charts(res2, 
               PARAS['excel_pth'], 
               '周数据', 
               ['red', 'red', 'red'],
               chart_type = 'column',
               index_name = '日期',
               gridline_type = None,
               format_list=['#,##0.00', '0', '#,##0.00'],
               workbook = wb)

一个df多个chart


### 3. '申请人RE-BLUE入会天数' 老会员，性别，年龄；

In [67]:
# 3. '申请人RE-BLUE入会天数' 老会员，性别，年龄；
tmp1['is_old'] = tmp1['申请人INMIX入会天数']>0
res3 = tmp1[['idcard_no', 'name', 'is_old', 'gender', '申请人年龄']].drop_duplicates()
old_res = res3.is_old.value_counts()
gender_res = res3.gender.value_counts()
res3 = pd.cut(res3['申请人年龄'], 
              bins=[0, 20, 30, 40, 50, 200], 
              labels=['20岁以下','21~30岁', '31~40岁', '41~50岁', '50岁以上']).value_counts()
res3 = res3.sort_index()
res3 = pd.DataFrame(res3)

wb, _ = df2charts(res3, 
               PARAS['excel_pth'], 
               '年龄分布', 
               ['red'],
               chart_type = 'column',
               index_name = '年龄',
               gridline_type = None,
               format_list=['0'],
               workbook = wb)

一个df多个chart


In [68]:
old_df = pd.DataFrame(old_res)
old_df.columns = ['人次']
old_df.index = old_df.index.map({False:'新顾客', True:'老顾客'})
wb, _ = df2charts(old_df, 
                PARAS['excel_pth'], 
               '顾客结构-饼图', 
               ['red'],
               chart_type = 'pie',
               index_name = '人次',
               add_chart_style='hh',
               gridline_type = None,
               format_list=['0'],
               workbook = wb)

一个df一个chart


In [69]:
gender_df = pd.DataFrame(gender_res)
gender_df.columns = ['人次']
gender_df.loc[1] = 0
gender_df.index = gender_df.index.map({'0':'女', 1:'男'})

wb, _ = df2charts(gender_df, 
               PARAS['excel_pth'], 
               '顾客性别-饼图', 
               ['red'],
               chart_type = 'pie',
               add_chart_style='hh',
               index_name = '性别',
               gridline_type = None,
               format_list=['0'],              
               workbook = wb)

一个df一个chart


### 4. 金额分布

In [70]:
# 4. 金额分布
res4 = tmp1[['idcard_no', 'name', 'goodsprice']]
res4 = res4.pivot_table(index='idcard_no', values='goodsprice', aggfunc='sum')
res4 = pd.cut(res4.goodsprice, 
              bins=[0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 1000000],
              labels=['￥0-1000', '￥1000-2000', '￥2000-3000', '￥3000-4000', '￥4000-5000', 
                      '￥5000-6000', '￥6000-7000', '￥7000-8000', '￥8000-9000', '￥9000以上']).value_counts()
res4 = res4.sort_index()
res4 = pd.DataFrame(res4)
res4.columns = ['人次']

In [71]:
wb, _ = df2charts(res4, 
               PARAS['excel_pth'], 
               '消费区间', 
               ['red'],
               chart_type = 'column',
               index_name = '价格',
               gridline_type = None,
               format_list=['0'],               
               workbook = wb)

一个df多个chart


### 5. 新老顾客分析

In [72]:
# 5. 新老顾客分析
# 总金额
res5 = tmp1.pivot_table(index='is_old', values=['goodsprice', 'num_orders'], aggfunc=sum)
res5['goodsprice_ratio'] = res5.goodsprice/res5.goodsprice.sum()
res5['meanprice'] = res5.goodsprice/res5.num_orders
res5.index = res5.index.map({False:'新会员', True:'老会员'})
res5 = res5[['goodsprice_ratio', 'meanprice']]
res5.columns = ['消费金额占比', '笔均金额']

In [73]:
wb, _ = df2charts(res5, 
               PARAS['excel_pth'], 
               '新老顾客', 
               ['red','black'],
               chart_type = 'column',
               index_name = '新老会员',
               gridline_type = None,
               format_list=['0.0%', '#,##0.00'],               
               workbook = wb)

一个df多个chart


### 6. 消费区间

In [74]:
# 6. 消费区间
res6 = tmp1[['goodsprice', 'is_old', 'num_orders']]
olds = res6[res6.is_old==True]
new = res6[res6.is_old==False]
new_res = pd.cut(new.goodsprice, 
                 bins=[0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 1000000],
                 labels=['￥0-1000', '￥1000-2000', '￥2000-3000', '￥3000-4000', '￥4000-5000', 
                      '￥5000-6000', '￥6000-7000', '￥7000-8000', '￥8000-9000', '￥9000以上']).value_counts()
old_res = pd.cut(olds.goodsprice, 
                 bins=[0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 1000000],
                 labels=['￥0-1000', '￥1000-2000', '￥2000-3000', '￥3000-4000', '￥4000-5000', 
                      '￥5000-6000', '￥6000-7000', '￥7000-8000', '￥8000-9000', '￥9000以上']).value_counts()
res6 = pd.DataFrame(columns=['新顾客人次','老顾客人次'])
res6['新顾客人次'] = new_res
res6['老顾客人次'] = old_res
res6 = res6.sort_index()

In [75]:
wb, _ = df2charts(res6, 
               PARAS['excel_pth'], 
               '新老顾客-消费区间', 
               ['red','black'],
               chart_type = 'column',
               index_name = '价格',
               gridline_type = None,
               format_list=['0', '0'],   
               add_chart_style='hh',
               workbook = wb)

一个df一个chart


### 7. 门店 金额，单数，单价

In [76]:
# 7. 门店 金额，单数，单价
res7 = tmp1[['goodsprice', 'storename', 'num_orders', 'fund_dt']]
res7 = res7.pivot_table(index='storename', values=['goodsprice', 'num_orders'], aggfunc=sum)
res7['meanprice'] = res7.goodsprice/res7.num_orders
res7.columns = ['交易金额', '交易单量', '单价']
res7 = res7.sort_values('交易金额', ascending=False)

In [77]:
wb, _ = df2charts(res7, 
               PARAS['excel_pth'], 
               '门店汇总', 
               ['red','red','red'],
               chart_type = 'bar',
               index_name = '门店',
               gridline_type = None,
               format_list=['#,##0.00', '0', '#,##0.00'],
               add_chart_style='new',
               workbook = wb)

一个df多个chart


In [78]:
wb.close()

### 8. 分门店数据

In [86]:
# 8. 分门店数据
res8 = tmp1[['idcard_no', 'status', 'goodsprice', 'storename', 'num_orders', 'week']].drop_duplicates()
#每一周数据都要有;
res8.index = range(len(res8))
for st in res8.storename.unique():
    for wk in ['第一周', '第二周', '第三周', '第四周']:
#     for wk in res8.week.unique():
        tmp = res8[(res8.storename==st) & (res8.week==wk)]
        if tmp.empty:
            new_record = {'idcard_no':'0000',
                          'status':'Funded',
                          'goodsprice':0,
                          'storename':st,
                          'num_orders':0,
                          'week':wk
                         }
            res8 = res8.append(new_record, ignore_index=True)

            
res8_1 = res8.pivot_table(index=['storename', 'week'], values=['goodsprice', 'num_orders'], aggfunc='sum')
res8_2 = res8.pivot_table(index=['storename'], values=['goodsprice', 'num_orders'], aggfunc='sum')
res8_2['mean'] = res8_2.goodsprice/res8_2.num_orders

In [87]:
for idx, st in enumerate(res8.storename.unique()):
    tmp_df = res8_1.loc[st].sort_index()
    tmp_df = tmp_df[['goodsprice']]
    smy_df = pd.DataFrame(res8_2.loc[st]).T
    print(st, tmp_df.shape, smy_df.shape)
    if idx==0:
        wb, ws = df2charts(tmp_df, 
                       PARAS['store_excel_pth'], 
                       st, 
                       ['red'],
                       chart_type = 'column',
                       index_name = '日期',
                       gridline_type = None,
                       format_list=['#,##0.00'])
        
        wb, _ = df2charts(smy_df, 
                       PARAS['store_excel_pth'], 
                       st, 
                       ['red', 'red', 'red'],
                       chart_type = 'column',
                       index_name = '店名',
                       gridline_type = None,
                       header_rownum = 9,
                       add_chart=False,
                       format_list=['#,##0.00', '0', '#,##0.00'],
                       workbook=wb,
                       worksheet=ws)
    else:
        wb, ws = df2charts(tmp_df, 
                       PARAS['store_excel_pth'], 
                       st, 
                       ['red'],
                       chart_type = 'column',
                       index_name = '日期',
                       gridline_type = None,
                       format_list=['#,##0.00'],
                       workbook=wb)
        
        wb, _ = df2charts(smy_df, 
                       PARAS['store_excel_pth'], 
                       st, 
                       ['red', 'red', 'red'],
                       chart_type = 'column',
                       index_name = '店名',
                       gridline_type = None,
                       header_rownum = 9,
                       add_chart=False,
                       format_list=['#,##0.00', '0', '#,##0.00'],
                       workbook=wb,
                       worksheet=ws)
wb.close()

INMIX_合生汇新店 (4, 1) (1, 3)
一个df多个chart
INMIX__INMIX_成都龙湖天街 (4, 1) (1, 3)
一个df多个chart
